### **Libraries**

Some of the needed libraries for this project.

In [20]:
import os
import openai
from openai import OpenAI

In [21]:
import pathlib
import textwrap

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [22]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # OpenAI API key

#### **Define a client**

Created an instance of the OpenAI model

In [23]:
# define an instance of OpenAI model
client = OpenAI(api_key=OPENAI_API_KEY)

#### **Audio transcription**

Transcribed the audio file.

In [44]:
audio_file= open("recent.mp3", "rb")

In [45]:
speech_to_text = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
)

In [46]:
to_markdown(speech_to_text.text)

> A list of the events I'll be having in the meantime. On the 8th of August, I'll be having a demo for PagDrop by 3pm. Set the reminder for 15 minutes. While on the 9th of August, I'll be having an interview with Google for the position of Data Scientist. The time is 7pm and the interview will end by 7.30pm. Set the reminder for 10 minutes. While on the 10th of August, I'll be having a final presentation with the university by 10am and it will last for an hour. So set the reminder for 5 minutes. While on the 11th of August, I'll be having another event which is Project Continuation. It's going to start by 6pm and it will end by 9pm. Set the reminder for 30 minutes.

### **Conversational chat**

Using conversational chat to extract event related data.

In [47]:
extract_prompt = f"""
You are an assistant that creates a well crafted schedule from text. 
Extract all events related data from the provided text and put it in JSON format.


{speech_to_text.text}
"""

In [48]:
from langchain_openai import ChatOpenAI

In [49]:
chatmodel = ChatOpenAI(
    model='gpt-4o',
    api_key=OPENAI_API_KEY,
    temperature=0.3,
)

In [50]:
format_chat  = chatmodel.invoke(extract_prompt)


In [51]:
to_markdown(format_chat.content)

> ```json
> {
>   "events": [
>     {
>       "title": "Demo for PagDrop",
>       "date": "2023-08-08",
>       "start_time": "15:00",
>       "end_time": null,
>       "reminder": "15 minutes"
>     },
>     {
>       "title": "Interview with Google for the position of Data Scientist",
>       "date": "2023-08-09",
>       "start_time": "19:00",
>       "end_time": "19:30",
>       "reminder": "10 minutes"
>     },
>     {
>       "title": "Final presentation with the university",
>       "date": "2023-08-10",
>       "start_time": "10:00",
>       "end_time": "11:00",
>       "reminder": "5 minutes"
>     },
>     {
>       "title": "Project Continuation",
>       "date": "2023-08-11",
>       "start_time": "18:00",
>       "end_time": "21:00",
>       "reminder": "30 minutes"
>     }
>   ]
> }
> ```

#### **Convert the datatype**

In [52]:
data_type = f"""
I want you to convert the date and time from the text to a format that can be easily converted
Put all time in 24 hours format.


Example:
21st of July 2024 >> 2024-07-21
10 minutes >> 00:10

and parse the output as a JSON object, just the JSON object of the event data and nothing else.


{format_chat.content}
"""

In [53]:
data_chat = chatmodel.invoke(data_type)
to_markdown(data_chat.content)

> ```json
> {
>   "events": [
>     {
>       "title": "Demo for PagDrop",
>       "date": "2023-08-08",
>       "start_time": "15:00",
>       "end_time": null,
>       "reminder": "00:15"
>     },
>     {
>       "title": "Interview with Google for the position of Data Scientist",
>       "date": "2023-08-09",
>       "start_time": "19:00",
>       "end_time": "19:30",
>       "reminder": "00:10"
>     },
>     {
>       "title": "Final presentation with the university",
>       "date": "2023-08-10",
>       "start_time": "10:00",
>       "end_time": "11:00",
>       "reminder": "00:05"
>     },
>     {
>       "title": "Project Continuation",
>       "date": "2023-08-11",
>       "start_time": "18:00",
>       "end_time": "21:00",
>       "reminder": "00:30"
>     }
>   ]
> }
> ```

#### **String output to JSON file**

In [54]:
# Used to create a JSON file
import json

In [60]:
data_chat.content

'```json\n{\n  "events": [\n    {\n      "title": "Demo for PagDrop",\n      "date": "2023-08-08",\n      "start_time": "15:00",\n      "end_time": null,\n      "reminder": "00:15"\n    },\n    {\n      "title": "Interview with Google for the position of Data Scientist",\n      "date": "2023-08-09",\n      "start_time": "19:00",\n      "end_time": "19:30",\n      "reminder": "00:10"\n    },\n    {\n      "title": "Final presentation with the university",\n      "date": "2023-08-10",\n      "start_time": "10:00",\n      "end_time": "11:00",\n      "reminder": "00:05"\n    },\n    {\n      "title": "Project Continuation",\n      "date": "2023-08-11",\n      "start_time": "18:00",\n      "end_time": "21:00",\n      "reminder": "00:30"\n    }\n  ]\n}\n```'

The format of the output of *data_chat.content* was re-formatted to enable the JSON object to be parsed properly. 

In [61]:
# converts string to dictionary
result_dict = json.loads(data_chat.content.strip('```json').strip('```'))

In [62]:

# Define the file path
json_file_path = 'events.json'

# Write the result to the JSON file
with open(json_file_path, 'w') as file:
    json.dump(result_dict, file, indent=4)

#### **JSON file to .ics file**

In [63]:
import json  # Import json module to read JSON file
from icalendar import Calendar, Event, Alarm
from datetime import datetime, timedelta
from pytz import UTC  # This is required for the 'dt' fields in icalendar

# Path to your JSON file
json_file_path = 'events.json'

# Read and load the JSON data from file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Create an iCalendar file
cal = Calendar()

for event in data['events']:
    # Create an event
    ical_event = Event()
    
    # Correctly access the properties of each event
    start_date_str = event['date'] + ' ' + event['start_time']
    end_date_str = event['date'] + ' ' + event['end_time']
    reminder_str = event['date'] + ' ' + event['reminder']
    
    # Convert to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    reminder_time = datetime.strptime(reminder_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    
    # Calculate reminder trigger time as a timedelta before the event start
    reminder_delta = start_date - reminder_time
    
    # Add details to the event
    ical_event.add('summary', event['event'])
    ical_event.add('dtstart', start_date)
    ical_event.add('dtend', end_date)
    
    # Optional: Add location and description if available
    if 'location' in event:
        ical_event.add('location', event['location'])
    if 'event_details' in event:
        ical_event.add('description', event.get('event_details', ''))

    # Create a reminder (alarm)
    alarm = Alarm()
    alarm.add('action', 'DISPLAY')
    alarm.add('description', 'Reminder')
    alarm.add('trigger', -reminder_delta)
    
    # Add the alarm to the event
    ical_event.add_component(alarm)
    
    # Add the event to the calendar
    cal.add_component(ical_event)

# Save the iCalendar file
file_name = 'events.ics'
with open(file_name, 'wb') as f:
    f.write(cal.to_ical())

print(f"iCalendar file '{file_name}' created successfully.")

TypeError: can only concatenate str (not "NoneType") to str

#### **Writing the .ics file to calendar**

In [ ]:
import os
from abc import ABC, abstractmethod
from icalendar import Calendar

# Step 1: Read and parse the ICS file
def read_ics(file_name):
    with open(file_name, 'r') as ics_file:
        return Calendar.from_ical(ics_file.read())

# Step 2: Define an abstract base class for calendar services
class CalendarService(ABC):
    def __init__(self, credentials):
        self.credentials = credentials

    @abstractmethod
    def authenticate(self):
        pass

    @abstractmethod
    def create_event(self, event):
        pass

# Step 3: Implement Google Calendar handler
class GoogleCalendarService(CalendarService):
    def authenticate(self):
        from google.oauth2.credentials import Credentials
        from google_auth_oauthlib.flow import InstalledAppFlow
        from google.auth.transport.requests import Request
        from googleapiclient.discovery import build

        SCOPES = ['https://www.googleapis.com/auth/calendar']
        
        creds = None
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials, SCOPES)
                creds = flow.run_local_server(port=0)
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
        
        self.service = build('calendar', 'v3', credentials=creds)

    def create_event(self, event):
        google_event = {
            'summary': str(event.get('summary')),
            'start': {'dateTime': event.get('dtstart').dt.isoformat(), 'timeZone': 'UTC'},
            'end': {'dateTime': event.get('dtend').dt.isoformat(), 'timeZone': 'UTC'}
        }
        self.service.events().insert(calendarId='primary', body=google_event).execute()

# Step 4: General function to add events to calendar
def add_events_to_calendar(service, ics_file_path):
    calendar = read_ics(ics_file_path)
    for component in calendar.walk():
        if component.name == "VEVENT":
            service.create_event(component)


# Initialize the Google Calendar service
google_service = GoogleCalendarService('credential.json')
google_service.authenticate()
add_events_to_calendar(google_service, 'events.ics')

RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})